In [12]:
# Do the necessary imports

import pickle 
import time
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print('setup complete')

setup complete


In [2]:
df = pd.read_csv('clean_df.csv')

In [3]:
#droping target classes

#df  = df[df['target']!='Bus']
#df  = df[df['target']!='Train']
#df  = df[df['target']!='Still']

In [4]:
df.target = df.target.replace({'Still':0, 'Walking':1, 'Car':2, 'Bus':3, 'Train':3})

In [5]:
df = df.drop([ 'speed_mean'], axis = 1)

In [6]:
df.head()

,android_sensor_gyroscope_mean,android_sensor_accelerometer_std,android_sensor_gyroscope_std,android_sensor_gyroscope_uncalibrated_mean,android_sensor_accelerometer_max,android_sensor_linear_acceleration_mean,android_sensor_rotation_vector_mean,android_sensor_rotation_vector_max,android_sensor_accelerometer_min,android_sensor_magnetic_field_uncalibrated_min,sound_min,target,user
0,0.001651,0.014626,0.000737,0.016221,9.849411,0.020978,0.050413,0.056351,9.758895,51.199707,83.859708,0,Luca
1,0.036326,1.775944,0.020290,0.039023,17.146631,0.879220,0.999981,0.999999,7.707437,82.409890,89.065143,2,Luca
2,0.001525,0.011199,0.000713,0.016302,9.849262,0.034110,0.610456,0.610456,9.804817,55.501802,83.859708,0,Luca
3,0.036440,0.862553,0.010553,0.050759,12.304298,1.488361,0.998112,0.998112,7.659674,95.664309,87.470377,2,Luca
4,0.183202,0.504117,0.098819,0.265652,10.891645,1.658308,0.322242,0.378193,8.965621,156.795909,89.770732,2,andrea


In [7]:
print(len(df))
df.nunique()

5893


android_sensor_gyroscope_mean                     5017
android_sensor_accelerometer_std                  5861
android_sensor_gyroscope_std                      4926
android_sensor_gyroscope_uncalibrated_mean        5001
android_sensor_accelerometer_max                  5838
android_sensor_linear_acceleration_mean           4972
android_sensor_rotation_vector_mean               5003
android_sensor_rotation_vector_max                4992
android_sensor_accelerometer_min                  5833
android_sensor_magnetic_field_uncalibrated_min    4708
sound_min                                         2305
target                                               4
user                                                13
dtype: int64

### We need to separate users from train and test. (we need atleast 1180 rows of data thats not in train)

#### Drop few users data such that the train data is 80% of original data

In [8]:
#df['user'].unique().tolist()

In [9]:
#TRAIN DATA

#drop these users from train data so that the model will never see them in test

luca_index = df[df['user'] == 'Luca'].index
Ivan_index = df[df['user'] == 'IvanHeibi'].index
Pier_index = df[df['user'] == 'Pierpaolo'].index
Riccardo_index = df[df['user'] == 'Riccardo'].index

# Delete these row indexes from dataFrame
train_df = df.drop(luca_index)
train_df = train_df.drop(Ivan_index)
train_df = train_df.drop(Pier_index)
train_df = train_df.drop(Riccardo_index)

print(len(train_df))
train_df.head()

4454


,android_sensor_gyroscope_mean,android_sensor_accelerometer_std,android_sensor_gyroscope_std,android_sensor_gyroscope_uncalibrated_mean,android_sensor_accelerometer_max,android_sensor_linear_acceleration_mean,android_sensor_rotation_vector_mean,android_sensor_rotation_vector_max,android_sensor_accelerometer_min,android_sensor_magnetic_field_uncalibrated_min,sound_min,target,user
4,0.183202,0.504117,0.098819,0.265652,10.891645,1.658308,0.322242,0.378193,8.965621,156.795909,89.770732,2,andrea
5,0.053366,0.103857,0.028408,0.060895,9.520044,0.471370,0.880361,0.904736,9.149029,134.198130,57.361127,2,Federica
6,0.004100,0.013489,0.001801,0.042991,9.816197,0.027441,0.201198,0.201318,9.758751,170.521377,65.158369,3,michelangelo
8,0.041554,0.255053,0.030074,0.065754,10.027802,0.278997,0.780795,0.786845,8.996134,134.611517,89.808441,3,andrea
9,0.037451,0.267791,0.021391,0.030491,10.068528,0.255172,0.968082,0.968806,9.014786,140.174223,89.815738,3,andrea


In [11]:
train_df.to_csv("model_train.csv", index = False)

In [ ]:
for name, group in train_df.groupby('user'):
    print(name,len(group))

In [ ]:
#TEST DATA

andrea_index = df[df['user'] == 'andrea'].index
federica_index = df[df['user'] == 'Federica'].index
michel_index = df[df['user'] == 'michelangelo'].index
damiano_index = df[df['user'] == 'Damiano'].index
claudio_index = df[df['user'] == 'Claudio'].index
vincenzo_index = df[df['user'] == 'Vincenzo'].index
serena_index = df[df['user'] == 'Serena'].index
andreacarpi_index = df[df['user'] == 'AndreaCarpineti'].index
elena_index = df[df['user'] == 'Elena'].index


# Delete these row indexes from dataFrame
test_df = df.drop(andrea_index)
test_df = test_df.drop(federica_index)
test_df = test_df.drop(michel_index)
test_df = test_df.drop(damiano_index)
test_df = test_df.drop(claudio_index)
test_df = test_df.drop(vincenzo_index)
test_df = test_df.drop(serena_index)
test_df = test_df.drop(andreacarpi_index)
test_df = test_df.drop(elena_index)


print(len(test_df))
test_df.head()

In [ ]:
#split the features and target in train data
print(train_df.user.unique())
print(test_df.user.unique())

X = train_df.drop(['target','user'], axis = 1)
y = train_df.target

print(len(X))
print(len(y))

In [ ]:
#split the features and target in test data


X_test = test_df.drop(['target','user'], axis = 1)
y_test = test_df.target

print(len(X_test))
print(len(y_test))

In [ ]:
X_test.head()

In [ ]:
print(X.nunique())
print(y.nunique())

In [ ]:
#numerical columns in the train data

num_vars  = list(X.columns)
num_vars

In [ ]:
# Pipeline for Tree models

num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(missing_values=np.nan ,strategy='mean')),
  ('Normalizer',RobustScaler())
  
])


tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

In [ ]:
#import the tress models

from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
#from xgboost               import XGBClassifier
#from lightgbm              import LGBMClassifier
#from catboost              import CatBoostClassifier

In [ ]:
#create a dict with the models that we want to train on

tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier()
  #"XGBoost":XGBClassifier(),
  #"LightGBM":LGBMClassifier(),
  #"CatBoost":CatBoostClassifier()
}

In [ ]:
#pass the pipeline to everymodel

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}
#tree_classifiers

In [ ]:
#fit the data to the model

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    X, y,
    test_size=0.2,
    stratify = y,   
    random_state=37
)

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(X_test)  
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)

    
results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

In [ ]:
results_ord.Accuracy.values[0]

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

print(confusion_matrix(y_test, pred))


In [ ]:


# # save the model to disk
# filename = 'final_model_v1.sav'
# pickle.dump(model, open(filename, 'wb'))
 
# # load the model from disk
# #loaded_model = pickle.load(open(filename, 'rb'))
# #result = loaded_model.score(X_test, Y_test)
# #print(result)